In [170]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from xml.etree import cElementTree as ET
pd.set_option('display.max_columns', None)

In [178]:
def get_location(latitud,longitud):
    if (latitud == 'nan') and (longitud == 'nan'):
        url = shop = road = city = suburb = region = state_district = state = postcode = country = np.nan
    else:
        url = 'https://nominatim.openstreetmap.org/reverse?lat={latitud}&lon={longitud}'.format(latitud = latitud, longitud = longitud)
        response = requests.get(url)
        root = ET.fromstring(response.text)
        try:
            shop            = root.find('addressparts').find('shop').text
        except:
            shop            = np.nan
        try:
            road            = root.find('addressparts').find('road').text
        except:
            road            = np.nan
        try:
            suburb          = root.find('addressparts').find('suburb').text
        except:
            suburb          = np.nan
        try:
            city            = root.find('addressparts').find('city').text
        except:
            city            = np.nan
        try:
            region          = root.find('addressparts').find('region').text
        except:
            region          = np.nan
        try:
            state_district  = root.find('addressparts').find('state_district').text
        except:
            state_district  = np.nan
        try:
            state           = root.find('addressparts').find('state').text
        except:
            state           = np.nan
        try:
            postcode        = root.find('addressparts').find('postcode').text
        except:
            postcode        = np.nan
        try:
            country         = root.find('addressparts').find('country').text
        except:
            country         = np.nan
    
    return(latitud,longitud,shop,road,suburb,city,region,state_district,state,postcode,country)

In [187]:
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim

nominatim = Nominatim()
areaId = nominatim.query('España').areaId()
overpass = Overpass()
query = overpassQueryBuilder(area=areaId, 
                                #elementType='area', 
                                elementType='node',
                                selector='"shop"="mall"', out='body')
result = overpass.query(query)
print(result.countElements())
result = pd.json_normalize( result.toJSON()['elements'])


overpass = Overpass()
query = overpassQueryBuilder(area=areaId, 
                                #elementType='area', 
                                elementType='node',
                                selector='"shop"="department_store"', out='body')
result2 = overpass.query(query,timeout=100)
print(result2.countElements())
result2 = pd.json_normalize( result2.toJSON()['elements'])


data = (
    pd.concat([result,result2],axis=0)
    .drop_duplicates('id')
    .reset_index(drop=True)
    [['id','lat','lon','tags.name','tags.postal_code','tags.addr:postcode']]
    .astype(str)
    )
data


138
277


,id,lat,lon,tags.name,tags.postal_code,tags.addr:postcode
0,253444189,38.3701678,-0.4650035,Parque Comercial Vistahermosa,nan,nan
1,254094451,38.3379544,-0.4864956,Panoramis,nan,nan
2,256322091,43.3899644,-5.8019998,C.C. Parque Principado,nan,nan
3,262959237,43.4383662,-3.8390288,El Corte Inglés,39011,nan
4,299661084,38.055388,-1.217246,Centro Comercial Vega Plaza,nan,nan
...,...,...,...,...,...,...
410,9202255468,42.2250959,2.8596305,Complements Parelols,nan,nan
411,9350656023,40.4731732,-3.4430071,Fnac,nan,nan
412,9372949221,37.1845419,-1.8207995,Tien 21,nan,04630
413,9455406917,42.8494997,-8.6743664,Comercio Rivas,nan,nan


In [189]:
res

,latitud,longitud,shop,road,suburb,city,region,state_district,state,postcode,country
0,38.3701678,-0.4650035,Parque Comercial Vistahermosa,avenida Antonio Ramos Carratalá,Vistahermosa,Alacant / Alicante,l'Alacantí,Alacant / Alicante,Comunitat Valenciana,03016,España
1,38.3379544,-0.4864956,NaN,calle Heliodoro Madrona,Puerto de Alicante,Alacant / Alicante,l'Alacantí,Alacant / Alicante,Comunitat Valenciana,03003,España
2,43.3899644,-5.8019998,Parque Principado,Carretera Oviedo - Viella,NaN,Siero,NaN,Asturias / Asturies,Asturias / Asturies,33429,España
3,43.4383662,-3.8390288,El Corte Inglés,Calle de Francisco Tomás y Valiente,Nueva Montaña,Santander,NaN,NaN,Cantabria,39011,España
4,38.055388,-1.217246,Centro Comercial Vega Plaza,Calle del Pensionista,NaN,NaN,NaN,Región de Murcia,Región de Murcia,30500,España
...,...,...,...,...,...,...,...,...,...,...,...
410,42.2250959,2.8596305,Complements Parelols,Carrer Olot,NaN,Navata,NaN,Girona,Catalunya,17744,España
411,40.4731732,-3.4430071,Centro Comercial Oasiz Madrid,Avenida de los Premios Nobel,NaN,Torrejón de Ardoz,NaN,NaN,Comunidad de Madrid,28859,España
412,37.1845419,-1.8207995,Tien 21,Calle Mayor,NaN,NaN,NaN,Almería,Andalucía,04630,España
413,42.8494997,-8.6743664,Comercio Rivas,Avenida dos Ánxeles,NaN,NaN,NaN,NaN,Galicia,15280,España


In [179]:
tqdm.pandas()
res = data.progress_apply(lambda x: get_location(x['lat'], x['lon']), axis=1)
res = pd.DataFrame(list(res), columns=['latitud','longitud','shop','road','suburb','city','region','state_district','state','postcode','country'])
res

100%|██████████| 415/415 [03:27<00:00,  2.00it/s]


0      (38.3701678, -0.4650035, Parque Comercial Vist...
1      (38.3379544, -0.4864956, nan, calle Heliodoro ...
2      (43.3899644, -5.8019998, Parque Principado, Ca...
3      (43.4383662, -3.8390288, El Corte Inglés, Call...
4      (38.055388, -1.217246, Centro Comercial Vega P...
                             ...                        
410    (42.2250959, 2.8596305, Complements Parelols, ...
411    (40.4731732, -3.4430071, Centro Comercial Oasi...
412    (37.1845419, -1.8207995, Tien 21, Calle Mayor,...
413    (42.8494997, -8.6743664, Comercio Rivas, Aveni...
414    (40.7692634, 0.5571052, GB Foods, C-42, nan, T...
Length: 415, dtype: object

In [188]:
res.query('city=="Malaga"')

,latitud,longitud,shop,road,suburb,city,region,state_district,state,postcode,country
